# Susceptibility & Resiliency — Starter Notebook

This notebook is a guided, step-by-step starting point for computing **susceptibility** and **resiliency** maps using raster inputs (DEM, slope, soil, drainage distance, landuse, population/building density).

Follow the cells in order. Each code cell includes comments explaining what it does.


## 1) Install required libraries (run once)

If you are using a conda environment or pip, install these packages. In Colab you can run `!pip install rasterio numpy matplotlib geopandas`.


In [7]:
# Install packages if needed (uncomment and run if required)
# !pip install rasterio numpy matplotlib geopandas rasterio[all] scikit-image

print('Skip installation step if packages are already installed.')

Skip installation step if packages are already installed.


## 2) Imports and helper functions

We'll write small helper functions to load rasters, normalize them, and perform the weighted overlay.

In [8]:

import os
import json
import numpy as np
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

def load_raster(path):
    ds = rasterio.open(path)
    arr = ds.read(1).astype('float32')
    arr[arr == ds.nodata] = np.nan
    return arr, ds.profile

def save_raster(arr, profile, out_path, nodata_value=-9999):
    profile2 = profile.copy()
    profile2.update(dtype=rasterio.float32, count=1, compress='lzw', nodata=nodata_value)
    arr2 = np.where(np.isnan(arr), nodata_value, arr).astype(rasterio.float32)
    with rasterio.open(out_path, 'w', **profile2) as dst:
        dst.write(arr2, 1)
    print('Saved', out_path)

def normalize(arr, method='minmax'):
    # arr may contain nan
    a = np.array(arr, dtype='float32')
    m = np.nanmin(a)
    M = np.nanmax(a)
    if np.isnan(m) or np.isnan(M) or M == m:
        return np.full_like(a, np.nan)
    return (a - m) / (M - m)


## 3) Example: compute susceptibility (weighted overlay)

Set paths to your raster layers and define weights. We normalize each raster (0-1) and combine with weights.

In [9]:

# === USER: set file paths here ===
# Replace these with your actual raster file paths (GeoTIFFs)
dem_fp = '/mnt/data/dem.tif'             # elevation (higher = less susceptible)
slope_fp = '/mnt/data/slope.tif'         # slope (higher slope = less susceptible in flood pooling)
soil_fp = '/mnt/data/soil.tif'           # soil permeability index (higher = less susceptible)
dist_drain_fp = '/mnt/data/dist_drain.tif' # distance to drainage (higher = less susceptible)
landuse_fp = '/mnt/data/landuse.tif'     # categorical; you'll need reclassification to susceptibility index

# Example weights (sum to 1.0)
sus_weights = {
    'dem': 0.30,
    'slope': 0.20,
    'soil': 0.20,
    'dist_drain': 0.15,
    'landuse': 0.15
}

def weighted_overlay_sus(paths, weights):
    arrays = {}
    profiles = {}
    for k, p in paths.items():
        if not os.path.exists(p):
            print(f'Warning: {p} not found. Skipping {k}.')
            arrays[k] = None
            profiles[k] = None
            continue
        arr, prof = load_raster(p)
        arrays[k] = arr
        profiles[k] = prof
    # Simple normalization and combination
    score = None
    for k, w in weights.items():
        arr = arrays.get(k)
        if arr is None:
            continue
        arr_n = normalize(arr)
        # For factors where higher value means less susceptibility, invert:
        if k in ('dem', 'slope', 'soil', 'dist_drain'):
            arr_n = 1 - arr_n  # so low elevation -> high susceptibility
        if score is None:
            score = np.zeros_like(arr_n, dtype='float32')
        score += np.nan_to_num(arr_n) * w
    return score, list(profiles.values())[0]

paths = {'dem': dem_fp, 'slope': slope_fp, 'soil': soil_fp, 'dist_drain': dist_drain_fp, 'landuse': landuse_fp}
sus_map, prof = weighted_overlay_sus(paths, sus_weights)
print('Computed susceptibility map (shape):', None if sus_map is None else sus_map.shape)

# Save example output (only if computed)
if sus_map is not None and prof is not None:
    outfp = '/mnt/data/susceptibility.tif'
    save_raster(sus_map, prof, outfp)


Computed susceptibility map (shape): None


## 4) Example: compute resiliency

Same idea but different weights and interpretation. Higher resiliency = better recovery (we'll produce resiliency score 0-1).

In [10]:

# === USER: set resiliency raster paths ===
building_fp = '/mnt/data/building_density.tif'
population_fp = '/mnt/data/pop_density.tif'
road_access_fp = '/mnt/data/road_access.tif'  # e.g., distance to nearest primary road (lower = better access)
drainage_eff_fp = '/mnt/data/drainage_eff.tif' # drainage efficiency index (higher = better)

res_weights = {
    'building': 0.25,
    'population': 0.20,
    'road_access': 0.20,
    'elevation': 0.15,
    'drainage_eff': 0.20
}

def weighted_overlay_res(paths, weights):
    arrays = {}
    profiles = {}
    for k, p in paths.items():
        if not os.path.exists(p):
            print(f'Warning: {p} not found. Skipping {k}.')
            arrays[k] = None
            profiles[k] = None
            continue
        arr, prof = load_raster(p)
        arrays[k] = arr
        profiles[k] = prof
    score = None
    for k, w in weights.items():
        arr = arrays.get(k)
        if arr is None:
            continue
        arr_n = normalize(arr)
        # For resiliency factors where higher is better (drainage_eff), keep as is.
        # For factors where higher raw value means worse resilience (population, building density, road distance),
        # invert so that higher normalized = more resilient
        if k in ('building', 'population', 'road_access'):
            arr_n = 1 - arr_n
        if score is None:
            score = np.zeros_like(arr_n, dtype='float32')
        score += np.nan_to_num(arr_n) * w
    return score, list(profiles.values())[0]

paths_res = {'building': building_fp, 'population': population_fp, 'road_access': road_access_fp, 'elevation': dem_fp, 'drainage_eff': drainage_eff_fp}
res_map, prof2 = weighted_overlay_res(paths_res, res_weights)
print('Computed resiliency map (shape):', None if res_map is None else res_map.shape)

if res_map is not None and prof2 is not None:
    outfp2 = '/mnt/data/resiliency.tif'
    save_raster(res_map, prof2, outfp2)


Computed resiliency map (shape): None


## 5) Visualize results

A simple plot to inspect the maps.

In [11]:

def quick_plot(arr, title=''):
    plt.figure(figsize=(6,4))
    plt.imshow(arr, interpolation='nearest')
    plt.colorbar()
    plt.title(title)
    plt.axis('off')
    plt.show()

if sus_map is not None:
    quick_plot(sus_map, 'Susceptibility score (0 low - 1 high)')
if res_map is not None:
    quick_plot(res_map, 'Resiliency score (0 low - 1 high)')


## 6) Save or export configuration (JSON)

Save the weights you used so the web frontend/backend can read them.

In [12]:

# config = {
#     "susceptibility": sus_weights,
#     "resiliency": res_weights,
#     "notes": "Weights sum to 1. Modify according to expert judgment or calibration."
# }
# with open('/mnt/data/config_weights.json', 'w', encoding='utf-8') as f:
#     json.dump(config, f, indent=2)
# print('Saved config to /mnt/data/config_weights.json')
# print(json.dumps(config, indent=2))

## 7) Next steps and calibration ideas

1. Reclassify categorical rasters (landuse, soil) into numeric vulnerability indices before normalization.
2. Use expert weighting or analytic hierarchy process (AHP) to set weights instead of guessing.
3. Validate results with historical flood extents (if available).
4. Consider sensitivity analysis: vary weights and measure change in high-risk areas.


# Dump


# Susceptibility

In [ ]:

# import os
# import numpy as np
# import rasterio
# from rasterio.warp import reproject, Resampling

# # Base directory 
# base_dir = r'C:\4TH YEAR\TERM 1\THS-ST2\Configuration'

# # Subfolders 
# folders = ['building_footprint', 'dem', 'drainage', 'flood_maps']

# # Helper functions 
# def find_raster_in_folder(folder_path):
#     """Find the first raster file (.tif, .tiff, .img) in a folder."""
#     if not os.path.exists(folder_path):
#         print(f"Folder not found: {folder_path}")
#         return None
#     for f in os.listdir(folder_path):
#         if f.lower().endswith(('.tif', '.tiff', '.img')):
#             return os.path.join(folder_path, f)
#     print(f"No raster found in: {folder_path}")
#     return None


# def load_raster(fp):
#     """Load raster as array and profile."""
#     with rasterio.open(fp) as src:
#         return src.read(1).astype('float32'), src.profile


# def normalize(arr):
#     """Normalize array to 0–1 scale."""
#     arr_min, arr_max = np.nanmin(arr), np.nanmax(arr)
#     if arr_max - arr_min == 0:
#         return np.zeros_like(arr)
#     return (arr - arr_min) / (arr_max - arr_min)


# def match_raster(ref_arr, ref_profile, arr, arr_profile):
#     """Resample arr to match the resolution, CRS, and transform of reference raster."""
#     dst = np.empty_like(ref_arr, dtype='float32')
#     reproject(
#         source=arr,
#         destination=dst,
#         src_transform=arr_profile['transform'],
#         src_crs=arr_profile['crs'],
#         dst_transform=ref_profile['transform'],
#         dst_crs=ref_profile['crs'],
#         resampling=Resampling.bilinear
#     )
#     return dst


# def save_raster(array, profile, output_path):
#     """Save array as GeoTIFF."""
#     profile.update(dtype=rasterio.float32, count=1)
#     with rasterio.open(output_path, 'w', **profile) as dst:
#         dst.write(array.astype(rasterio.float32), 1)


# # Automatically locate raster files 
# paths = {}
# for f in folders:
#     folder_path = os.path.join(base_dir, f)
#     raster_fp = find_raster_in_folder(folder_path)
#     if raster_fp:
#         paths[f] = raster_fp
#         print(f"Found raster for {f}: {raster_fp}")
#     else:
#         print(f"Skipping {f} (no file found).")

# if not paths:
#     raise FileNotFoundError("No raster files found. Please check your folder paths.")

# # Weights (must sum to 1.0)
# sus_weights = {
#     'dem': 0.4,
#     'drainage': 0.3,
#     'flood_maps': 0.2,
#     'building_footprint': 0.1
# }


# # Weighted overlay computation 
# def weighted_overlay_sus(paths, weights):
#     arrays, profiles = {}, {}

#     # Load all rasters
#     for k, p in paths.items():
#         arr, prof = load_raster(p)
#         arrays[k] = arr
#         profiles[k] = prof

#     # Pick the first raster as reference
#     ref_key = next(iter(arrays))
#     ref_arr = arrays[ref_key]
#     ref_prof = profiles[ref_key]

#     score = np.zeros_like(ref_arr, dtype='float32')

#     # Combine layers using weights
#     for k, w in weights.items():
#         arr = arrays.get(k)
#         prof = profiles.get(k)
#         if arr is None or prof is None:
#             continue
#         # Resample to match reference raster
#         arr_resampled = match_raster(ref_arr, ref_prof, arr, prof)
#         arr_n = normalize(arr_resampled)

#         # Invert if higher value = less susceptible
#         if k in ('dem', 'drainage'):
#             arr_n = 1 - arr_n

#         score += np.nan_to_num(arr_n) * w

#     return score, ref_prof


# # Run computation 
# sus_map, prof = weighted_overlay_sus(paths, sus_weights)
# print("Computed susceptibility map with shape:", sus_map.shape)

# # Save output 
# output_fp = os.path.join(base_dir, "susceptibility_map.tif")
# save_raster(sus_map, prof, output_fp)
# print(f"Saved susceptibility map to: {output_fp}")



In [ ]:
# === Susceptibility Configuration - Complete & Verified ===
import os
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt

# ---- 1️⃣ Base directory & folders ----
base_dir = r'C:\4TH YEAR\TERM 1\THS-ST2\Configuration'
folders = ['building_footprint', 'dem', 'drainage', 'flood_maps']

# ---- 2️⃣ Helper functions ----
def find_raster_in_folder(folder_path):
    """Find the first raster file (.tif, .tiff, .img) in a folder."""
    if not os.path.exists(folder_path):
        print(f"⚠️ Folder not found: {folder_path}")
        return None
    for f in os.listdir(folder_path):
        if f.lower().endswith(('.tif', '.tiff', '.img')):
            return os.path.join(folder_path, f)
    print(f"⚠️ No raster found in: {folder_path}")
    return None


def load_raster(fp):
    """Load raster as array and profile."""
    with rasterio.open(fp) as src:
        return src.read(1).astype('float32'), src.profile


def normalize(arr):
    """Normalize array to 0–1 scale."""
    arr_min, arr_max = np.nanmin(arr), np.nanmax(arr)
    if arr_max - arr_min == 0:
        return np.zeros_like(arr)
    return (arr - arr_min) / (arr_max - arr_min)


def match_raster(ref_arr, ref_profile, arr, arr_profile):
    """Resample arr to match the resolution, CRS, and transform of reference raster."""
    dst = np.empty_like(ref_arr, dtype='float32')
    reproject(
        source=arr,
        destination=dst,
        src_transform=arr_profile['transform'],
        src_crs=arr_profile['crs'],
        dst_transform=ref_profile['transform'],
        dst_crs=ref_profile['crs'],
        resampling=Resampling.bilinear
    )
    return dst


def save_raster(array, profile, output_path):
    """Save array as GeoTIFF."""
    profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(array.astype(rasterio.float32), 1)


# ---- 3️⃣ Automatically locate raster files ----
paths = {}
for f in folders:
    folder_path = os.path.join(base_dir, f)
    raster_fp = find_raster_in_folder(folder_path)
    if raster_fp:
        paths[f] = raster_fp
        print(f"✅ Found raster for {f}: {raster_fp}")
    else:
        print(f"❌ Skipping {f} (no file found).")

if not paths:
    raise FileNotFoundError("No raster files found. Please check your folder paths.")


# ---- 4️⃣ Define weights (sum to 1.0) ----
sus_weights = {
    'dem': 0.4,
    'drainage': 0.3,
    'flood_maps': 0.2,
    'building_footprint': 0.1
}

assert abs(sum(sus_weights.values()) - 1.0) < 1e-6, "❌ Weights must sum to 1.0"
print("✅ Weights verified (sum = 1.0)")


# ---- 5️⃣ Print CRS and shapes before alignment ----
print("\n🔍 Checking raster CRS and shapes:")
for name, p in paths.items():
    with rasterio.open(p) as src:
        print(f"{name:20s} | CRS: {src.crs} | Shape: {src.read(1).shape}")


# ---- 6️⃣ Weighted overlay computation ----
def weighted_overlay_sus(paths, weights):
    arrays, profiles = {}, {}

    # Load all rasters
    for k, p in paths.items():
        arr, prof = load_raster(p)
        arrays[k] = arr
        profiles[k] = prof

    # Use DEM as reference
    ref_key = 'dem' if 'dem' in arrays else next(iter(arrays))
    ref_arr = arrays[ref_key]
    ref_prof = profiles[ref_key]
    print(f"\n🗺️ Using {ref_key.upper()} as reference raster for alignment.")

    score = np.zeros_like(ref_arr, dtype='float32')

    # Combine layers using weights
    for k, w in weights.items():
        arr = arrays.get(k)
        prof = profiles.get(k)
        if arr is None or prof is None:
            print(f"⚠️ Skipping {k}: missing data")
            continue
        # Resample to match reference raster
        arr_resampled = match_raster(ref_arr, ref_prof, arr, prof)
        arr_n = normalize(arr_resampled)

        # Invert if higher value = less susceptible
        if k in ('dem', 'drainage'):
            arr_n = 1 - arr_n

        score += np.nan_to_num(arr_n) * w

    return score, ref_prof


# ---- 7️⃣ Run computation ----
sus_map, prof = weighted_overlay_sus(paths, sus_weights)
print("\n✅ Computed susceptibility map with shape:", sus_map.shape)


# ---- 8️⃣ Visualization of normalized layers ----
plt.figure(figsize=(12, 8))
for i, (k, p) in enumerate(paths.items(), 1):
    arr, prof = load_raster(p)
    arr_res = match_raster(sus_map, prof, arr, prof)
    arr_n = normalize(arr_res)
    if k in ('dem', 'drainage'):
        arr_n = 1 - arr_n
    plt.subplot(2, 2, i)
    plt.imshow(arr_n, cmap='viridis')
    plt.title(f"{k} (normalized)")
    plt.axis('off')
plt.tight_layout()
plt.show()


# ---- 9️⃣ Visualization of final susceptibility map ----
plt.figure(figsize=(8,6))
plt.imshow(sus_map, cmap='inferno')
plt.colorbar(label='Flood Susceptibility Index')
plt.title('Final Susceptibility Map')
plt.axis('off')
plt.show()


# ---- 🔟 Save output ----
output_fp = os.path.join(base_dir, "susceptibility_map.tif")
save_raster(sus_map, prof, output_fp)
print(f"💾 Saved susceptibility map to: {output_fp}")


# Resiliency


In [ ]:
# === CONFIGURATION AND FILE LOADING ===
import os
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show
from skimage import io

# === DEFINE FOLDERS ===
folders = {
    "building_footprint": "building_footprint",
    "dem": "dem",
    "drainage": "drainage",
    "flood_maps": "flood_maps"
}

# === AUTOMATIC FILE DISCOVERY ===
data_files = {}

for name, folder in folders.items():
    path = os.path.join(os.getcwd(), folder)
    if os.path.exists(path):
        files = [os.path.join(path, f) for f in os.listdir(path) if not f.startswith('.')]
        data_files[name] = files
    else:
        print(f"⚠️ Folder not found: {folder}")

# === DISPLAY FOUND FILES ===
print("📂 Loaded folders and files:")
for key, files in data_files.items():
    print(f"\n📁 {key}:")
    for f in files:
        print("   -", os.path.basename(f))

# === EXAMPLE: LOAD FIRST FILES FROM EACH FOLDER ===
print("\n✅ Attempting to load sample files...\n")

loaded_data = {}

for key, files in data_files.items():
    if not files:
        continue
    sample_file = files[0]
    print(f"➡️ Loading {key} sample: {os.path.basename(sample_file)}")

    try:
        if sample_file.endswith(('.shp', '.geojson')):
            loaded_data[key] = gpd.read_file(sample_file)
            print(f"   🗺️ Loaded vector data with {len(loaded_data[key])} features.")
        elif sample_file.endswith(('.tif', '.tiff')):
            loaded_data[key] = rasterio.open(sample_file)
            print(f"   🌄 Loaded raster data ({loaded_data[key].width}x{loaded_data[key].height}).")
        elif sample_file.endswith(('.png', '.jpg', '.jpeg')):
            loaded_data[key] = io.imread(sample_file)
            print(f"   🖼️ Loaded image with shape {loaded_data[key].shape}.")
        else:
            print(f"   ⚠️ Unsupported file format: {sample_file}")
    except Exception as e:
        print(f"   ❌ Error loading {sample_file}: {e}")

# === OPTIONAL: QUICK VISUALIZATION ===
for key, data in loaded_data.items():
    print(f"\n🧭 Preview of {key} data:")
    if isinstance(data, gpd.GeoDataFrame):
        data.plot(figsize=(6,6))
        plt.title(f"{key} - Vector Preview")
        plt.show()
    elif isinstance(data, rasterio.io.DatasetReader):
        show(data)
        plt.title(f"{key} - Raster Preview")
        plt.show()
    elif isinstance(data, np.ndarray):
        plt.imshow(data)
        plt.title(f"{key} - Image Preview")
        plt.axis('off')
        plt.show()
